# Setup

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import calendar
import time
import requests
from typing import List, Union

In [2]:
languages = ["eng", "English"]
with open('../LOOKUP-LANGUAGES.TXT', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            code, name = line.split('\t')  # Split by tab character
            languages.append(code)
            languages.append(name) 

print(languages)

['eng', 'English', 'afr', 'Afrikaans', 'sqi', 'Albanian', 'ara', 'Arabic', 'hye', 'Armenian', 'axe', 'Azerbaijani', 'ben', 'Bengali', 'bos', 'Bosnian', 'bul', 'Bulgarian', 'cat', 'Catalan', 'zho', 'Chinese', 'hrv', 'Croatian', 'ces', 'Czech', 'dan', 'Danish', 'nld', 'Dutch', 'est', 'Estonian', 'fin', 'Finnish', 'fra', 'French', 'glg', 'Galician', 'kat', 'Georgian', 'deu', 'German', 'ell', 'Greek', 'guj', 'Gujarati', 'heb', 'Hebrew', 'hin', 'Hindi', 'hun', 'Hungarian', 'isl', 'Icelandic', 'ind', 'Indonesian', 'ita', 'Italian', 'jpn', 'Japanese', 'kan', 'Kannada', 'kaz', 'Kazakh', 'kor', 'Korean', 'lav', 'Latvian', 'lit', 'Lithuanian', 'mkd', 'Macedonian', 'msa', 'Malay', 'mal', 'Malayalam', 'mar', 'Marathi', 'mon', 'Mongolian', 'nep', 'Nepali', 'nor', 'Norwegian', 'nno', 'NorwegianNynorsk', 'fas', 'Persian', 'pol', 'Polish', 'por', 'Portuguese', 'pan', 'Punjabi', 'ron', 'Romanian', 'rus', 'Russian', 'srp', 'Serbian', 'sin', 'Sinhalese', 'slk', 'Slovak', 'slv', 'Slovenian', 'som', 'Somal

# Article Scraper

Testing out manual fetching of articles

In [110]:
# query_string = "NVIDIA NVIDIA Corporation (environment OR social OR governance OR sustainability OR ESG rating OR climate change OR diversity OR emissions OR resource use OR corporate social responsibility) sourcelang:english"
# query_string = '("Fastenal" OR "Fastenal Company") ("ESG score" OR "ESG rating" OR ESG)'
query_string = '(FAST OR "Fastenal Company" OR "Fastenal Company") (environment OR social OR governance OR sustainability OR "ESG score" OR "ESG rating" OR ESG) ("climate change" OR diversity OR emissions OR "resource use") sourcelang:English'
mode = "artlist"
headers = {"User-Agent": f"GDELT Python API"}

In [111]:
response = requests.get(
            f"https://api.gdeltproject.org/api/v2/doc/doc?query={query_string}&mode={mode}&format=json&maxrecords=200&sourcelang=ara&startdatetime=20180101000000&enddatetime=20180131235959",
            headers=headers
        )

In [112]:
response.text

'{"articles": [ { "url": "http://www.moneycontrol.com/news/trends/expert-columns-2/value-investing-esg-linked-investments-in-india-likely-to-touch-240-billion-in-10-years-2491829.html", "url_mobile": "http://www.moneycontrol.com/news/trends/expert-columns-2/value-investing-esg-linked-investments-in-india-likely-to-touch-240-billion-in-10-years-2491829.html/amp", "title": "Value Investing ! ESG linked investments in India likely to touch $240 billion in 10 years", "seendate": "20180126T101500Z", "socialimage": "https://static-news.moneycontrol.com/static-mcnews/2017/04/shutterstock_102166600-770x433.jpg", "domain": "moneycontrol.com", "language": "English", "sourcecountry": "India" },{ "url": "https://dailytimes.com.pk/190327/development-sustainable-environment-pakistan-literature-review-part-ii/", "url_mobile": "https://dailytimes.com.pk/190327/development-sustainable-environment-pakistan-literature-review-part-ii/amp/", "title": "Development of sustainable environment in Pakistan : a 

In [113]:
pd.DataFrame(response.json(strict=False)['articles'])

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,http://www.moneycontrol.com/news/trends/expert...,http://www.moneycontrol.com/news/trends/expert...,Value Investing ! ESG linked investments in In...,20180126T101500Z,https://static-news.moneycontrol.com/static-mc...,moneycontrol.com,English,India
1,https://dailytimes.com.pk/190327/development-s...,https://dailytimes.com.pk/190327/development-s...,Development of sustainable environment in Paki...,20180128T011500Z,https://dailytimes.com.pk/assets/uploads/2017/...,dailytimes.com.pk,English,Pakistan
2,https://www.forbes.com/sites/christopherskroup...,https://www.forbes.com/sites/christopherskroup...,What The Value Where ESG Meets Emerging Market...,20180109T171500Z,https://blogs-images.forbes.com/christopherskr...,forbes.com,English,United States
3,http://reneweconomy.com.au/change-capitalism-b...,,We have to change capitalism to beat climat...,20180125T040000Z,http://reneweconomy.com.au/wp-content/uploads/...,reneweconomy.com.au,English,Australia
4,https://climatecrocks.com/2018/01/25/uber-capi...,https://climatecrocks.com/2018/01/25/uber-capi...,Uber Capitalist : We Have to Change Capitalism...,20180125T184500Z,https://climatecrock.files.wordpress.com/2015/...,climatecrocks.com,English,United States
...,...,...,...,...,...,...,...,...
195,http://newsroomamerica.com/story/680751.html,,NewsRoomAmerica . com,20180125T120000Z,,newsroomamerica.com,English,United States
196,http://www.disasternews.net/news/article.php?a...,,2014 - 16 sees record temp surge,20180127T093000Z,,disasternews.net,English,
197,https://www.mindbodygreen.com/articles/mootral...,https://amp.mindbodygreen.com/articles/mootral...,Why Cows Need Supplements,20180103T130000Z,https://mindbodygreen-res.cloudinary.com/image...,mindbodygreen.com,English,
198,http://www.theloop.ca/thanks-climate-change-ch...,,"Thanks to climate change , chocolate might dis...",20180103T221500Z,http://www.theloop.ca/wp-content/uploads/2018/...,theloop.ca,English,


Writing methods to automate the article fetcher

In [4]:
Filter = Union[List[str], str]
FilterGroup = List[List[str]]

In [5]:
def query_maker(companies: FilterGroup, keywords: FilterGroup, language=None):
    query_string = ""

    # process companies (assuming all companies are mandatory)
    # if type(companies) == str:
    #     query_string += f'"{companies} "' if " " in companies else f"{companies} "
    # else:
    #     query_string = query_string + " ".join([f'"{company}"' if " " in company else company for company in companies]) + " " # use space seperation for AND statements

    for group in companies:
        if not group: # skip empty groups
            continue
        # join groups with multiple companies with OR
        if len(group) > 1: 
            group_string = "(" + " OR ".join([f'"{company}"' if " " in company else company for company in group]) + ")"
        else:
            company = group[0]
            group_string = f'"{company}" ' if " " in company else f"{company}"

        query_string += group_string + " "

    # process keywords (assuming all keywords are interchangeable)
    # if type(keywords) == str:
    #     query_string += f'"{keywords} "'
    # else:
    #     query_string = query_string + "(" + " OR ".join([f'"{keyword}"' if " " in keyword else keyword for keyword in keywords]) + ") "

    # query_string += " "

    for group in keywords:
        if not group:  # skip empty groups
            continue
        # join groups with multiple keywords with OR 
        if len(group) > 1:
            group_string = "(" + " OR ".join([f'"{keyword}"' if " " in keyword else keyword for keyword in group]) + ")"
        else:
            keyword = group[0]
            group_string = f'"{keyword}" ' if " " in keyword else f"{keyword}"
            
        query_string += group_string + " "

    # process language
    if language:
        if language in languages:
            query_string += f"sourcelang:{language} "
        else:
            print("Language not found in lookup table")
    
    return query_string

In [136]:
# companies = [["NVIDIA", "NVIDIA Corporation"]]
keywords = [["environment", "social", "governance", "sustainability", "ESG score", "ESG rating", "ESG"], ["climate change", "diversity", "emissions", "resource use", 
                                                                                                            #  "corporate social responsibility"
                                                                                                             
                                                                                                             ]]

In [157]:
def query(query_string, mode, headers, start_date=None, end_date=None, max_records=250, ticker=None):

    
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")  
    start_dt = start_dt.strftime("%Y%m%d%H%M%S")

    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    end_dt = end_dt.replace(hour=23, minute=59, second=59)
    end_dt = end_dt.strftime("%Y%m%d%H%M%S")
    # end_date = "20180131235959"
    
    response = requests.get(
            f"https://api.gdeltproject.org/api/v2/doc/doc?query={query_string}&mode={mode}&format=json&maxrecords={max_records}&startdatetime={start_dt}&enddatetime={end_dt}",
            headers=headers
        )
    
    try:
        data = pd.DataFrame(response.json(strict=False)['articles'])
        if ticker:
            data.insert(0, "symbol", ticker)
        return data
    except requests.exceptions.JSONDecodeError:
        print(f"JSONDecodeError: {response.text}")
        return pd.DataFrame()
    except KeyError:
        # print(f"KeyError: {response.json()}")
        return pd.DataFrame()
    # return response

In [7]:
data = pd.read_csv("../data/processed/numerical.csv")

In [82]:
companies = data.drop_duplicates(subset=['symbol'])[['symbol', 'shortName', 'longName']].astype(str).values.tolist()

In [143]:
companies[0][1:]

['Fastenal Company', 'Fastenal Company']

In [154]:
companies[200:230]

[['TTKOM.IS', 'TURK TELEKOM', 'Türk Telekomünikasyon Anonim Sirketi'],
 ['INTP.JK',
  'Indocement Tunggal Prakarsa Tbk',
  'PT Indocement Tunggal Prakarsa Tbk'],
 ['7013.T', 'IHI CORPORATION', 'IHI Corporation'],
 ['0NOF.IL',
  'PROCTER & GAMBLE CO PROCTER & G',
  'The Procter & Gamble Company'],
 ['0QCV.IL', 'ABBVIE INC ABBVIE ORD SHS', 'AbbVie Inc.'],
 ['QIBK.QA', 'Qatar Islamic Bank QPSC', 'Qatar Islamic Bank (Q.P.S.C.)'],
 ['4507.T', 'SHIONOGI & CO', 'Shionogi & Co., Ltd.'],
 ['BPCL.NS',
  'BHARAT PETROLEUM CORP  LT',
  'Bharat Petroleum Corporation Limited'],
 ['AVIA.JK', 'Avia Avian Tbk.', 'PT Avia Avian Tbk'],
 ['7974.T', 'NINTENDO CO LTD', 'Nintendo Co., Ltd.'],
 ['0A2A.IL', 'STILLFRONT GROUP AB (PUBL) STIL', 'Stillfront Group AB (publ)'],
 ['CPRT.MX', 'COPART INC', 'Copart, Inc.'],
 ['PLD', 'Prologis, Inc.', 'Prologis, Inc.'],
 ['SCBFF', 'Standard Chartered PLC', 'Standard Chartered PLC'],
 ['CRL',
  'Charles River Laboratories Inte',
  'Charles River Laboratories Internationa

In [83]:
query_maker([companies[0]], keywords, "eng")

'(FAST OR "Fastenal Company" OR "Fastenal Company") (environment OR social OR governance OR sustainability) ("ESG score" OR "ESG rating" OR ESG) ("climate change" OR diversity OR emissions OR "resource use" OR "corporate social responsibility") sourcelang:eng '

In [158]:
%%time
dataframes = []
# company_df = []

for company in companies:
    # for i in range(12):
        # start_date = f"2024-{i+1}-01"

        # end_date = datetime.strptime(f"2024-{i+1}-01", "%Y-%m-%d")
        # last_day = calendar.monthrange(end_date.year, end_date.month)[1] # makes sure the last day of the given month is used
        # end_date = end_date.replace(day=last_day).strftime("%Y-%m-%d")
    print(f"Fetching data for {company[0]}")
    query_string = query_maker([company[1:]], keywords, language="English")
    dataframes.append(query(query_string, mode, headers, "2024-01-01", "2024-12-31", max_records=50, ticker=company[0])) 
    # print(f"Fetched data for {start_date} to {end_date}")
    time.sleep(1) # wait for 5 seconds to avoid overloading API calls

    # dataframes.append(pd.concat(company_df))
    # print(f"Concatenated data for {company[0]}")

news_df = pd.concat(dataframes)

Fetching data for FAST
Fetching data for 0A2S.IL
Fetching data for BXSL
Fetching data for NED.JO
Fetching data for SCBGF
Fetching data for SMAWF
Fetching data for APP.MX
Fetching data for 7911.T
Fetching data for BAJAJFINSV.NS
Fetching data for 7267.T
Fetching data for DTE.PR
Fetching data for 0O87.IL
JSONDecodeError: Your query was too short or too long.

Fetching data for MO
Fetching data for HCA.MX
Fetching data for ORDS.QA
Fetching data for QNDPF
JSONDecodeError: Your query was too short or too long.

Fetching data for NPH.JO
Fetching data for AFE.JO
Fetching data for UNVR.JK
Fetching data for 9602.T
Fetching data for GLW
Fetching data for LHC.JO
Fetching data for MPNGF
Fetching data for 6383.T
Fetching data for DCI
Fetching data for SCMA.JK
Fetching data for 6506.T
Fetching data for BCI.SN
Fetching data for 5929.T
Fetching data for MV91.F
Fetching data for ABLZF
Fetching data for SYY
Fetching data for 4151.T
Fetching data for ADANIPORTS.NS
JSONDecodeError: One or more of your pare

In [159]:
news_df

,symbol,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,FAST,https://www.wallstreet-online.de/nachricht/186...,,Fastenal Names New Executive Vice President of...,20241018T001500Z,https://assets.wallstreet-online.de/_media/6/s...,wallstreet-online.de,English,Germany
0,SMAWF,https://www.pandct.com/news/siemens-achieves-s...,,Process and Control Today | Siemens achieves s...,20241209T063000Z,http://www.pandct.com/uploads/news_images/Siem...,pandct.com,English,United Kingdom
1,SMAWF,https://www.webwire.com/ViewPressRel.asp?aId=3...,,Siemens to deploy technology at 60 government ...,20241122T211500Z,https://photos.webwire.com/prmedia/6/330111/33...,webwire.com,English,United States
2,SMAWF,https://www.prnewswire.com/news-releases/autom...,,Automation Solutions for Renewable Power Gener...,20241014T230000Z,,prnewswire.com,English,United States
3,SMAWF,https://menafn.com/1108777919/Automation-Solut...,,Automation Solutions For Renewable Power Gener...,20241014T223000Z,https://menafn.com/updates/pr/2024-10/14/PN_94...,menafn.com,English,Qatar
...,...,...,...,...,...,...,...,...,...
2,8411.T,https://www.thehindubusinessline.com/companies...,,"Today Business News Live Updates November 30 ,...",20241130T101500Z,https://bl-i.thgim.com/public/incoming/dxyssh/...,thehindubusinessline.com,English,India
3,8411.T,https://www.thehindubusinessline.com/companies...,,Today Business News Live : Domestic passenger ...,20241201T164500Z,https://bl-i.thgim.com/public/incoming/dxyssh/...,thehindubusinessline.com,English,India
4,8411.T,https://energycentral.com/c/ec/doe-awards-six-...,,DOE Awards Six HALEU Fuel Contracts for Advanc...,20241013T000000Z,,energycentral.com,English,United States
5,8411.T,https://www.thehindubusinessline.com/companies...,,Today Business News Live : Domestic passenger ...,20241202T071500Z,https://bl-i.thgim.com/public/incoming/dxyssh/...,thehindubusinessline.com,English,India


In [160]:
news_df.to_csv("../data/raw/news.csv", index=False)